In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = 15

In [2]:
df_nil_individuals=pd.read_excel(r'C:/Users/alruiz2201/Downloads/ContinuousIndNilViewers.xls') 

In [3]:
#In this cell the header of the report is Deleted, and the Columns are established.

rows_drop = list(range(0,8)) #An array is created that represents the number of rows that contains the header
df_nil_individuals=df_nil_individuals.drop(rows_drop, axis=0).drop(['Continuous Nil Viewers Individuals'],axis=1).reset_index(drop=True)# The header is deleted as well\
# as one false column 'Continiuous Nil......' and the index is reseted.
df_nil_individuals=df_nil_individuals.rename(columns=df_nil_individuals.iloc[0]).drop(df_nil_individuals.index[0]) #Rename the false columns with their correct names.
df_nil_individuals=df_nil_individuals.drop(index=df_nil_individuals.index[-1],axis=0) #Deletes the last row which is not important
df_nil_individuals=df_nil_individuals.replace(r'^\s*$', np.nan, regex=True) #All the blank cells are replaced for a NaN Value
df_nil_individuals=df_nil_individuals.iloc[:,:-1] #Drops the last column which is 'Household Code' and is duplicated

#Note: The Columns With Name 'NaN' are not Deleted, in order to not lose information.They will be dropped in the next lines

In [4]:
#Get the number of rows which does not have a value in the column 'Household Code' is obtained 
df_null_hhcode=df_nil_individuals[pd.isna(df_nil_individuals['Household Code'])]
df_null_hhcode_num=df_null_hhcode.shape[0]

In [5]:
#This cell reorganizes the values that are separated in the different rows into one to get the DF completly normal
for x in range(df_null_hhcode_num):
    df_null=df_nil_individuals[pd.isna(df_nil_individuals['Household Code'])] #Find the rows which are 'Nan' in the 'Household Code' Column/
     #Find the values that are null in the column "Household Code", and returns a dataframe with only this rows
    index_number=df_null.index[0] #Extracts the first element of the index array in the 'df_null' dataframe

    is_integer_area=isinstance(df_null.iloc[0][ 'Area'],int) #Verifies if the first element of the 'df_null' DF is\
    # of type 'int' in the column 'Area'
    is_integer_sd=isinstance(df_null.iloc[0]['Status Description'],int) #Verifies if the first element of the 'df_null'\ 
    # DF is of type 'int' in the column 'Status Description'
    if (is_integer_area == True or is_integer_sd==True):#If one of these conditions is true, it means that the entire row is defased one column to the left. 
        df_null.iloc[0]=df_null.iloc[0].shift(1) #Moves the entire row one column to the right 

    row_1=df_nil_individuals.loc[index_number-1, :] #Selects the row where we want to set the values
    row_2=df_null.iloc[0] #selects the first entire row of the "df_null" dataframe
    combined_row=row_1.combine_first(row_2) #Make the combination to set the combination between the two rows.

    df_nil_individuals.loc[index_number-1, :]=combined_row #The Combined Row is asigned to the row above of the row with the NaN Values.
    df_nil_individuals=df_nil_individuals.drop(index=index_number, axis=0).reset_index(drop=True)#The Row with the NaN value is deleted, now that the combination is succesfully

df_nil_individuals = df_nil_individuals.filter(regex='(?i)^(?!NaN).+', axis=1) #The Columns, with the name NaN are deleted,due to that all the info is organized\
# and these columns dont contain any data. 


In [6]:
#This cell of code joins many Rows into one Row. Which have in common the 'Household Id' and the 'Individual Id'.\
# All these new rows will be stored in a Array called A  
A=[]
non_repeated_hh=df_nil_individuals['Household Code'].unique().tolist() #Creates an array with the 'Household Code' elements
for x in non_repeated_hh:
    hh_group=df_nil_individuals.groupby('Household Code')
    hh_group=hh_group.get_group(x) #Get the 'x' household data
    non_repeated_id=hh_group['Individual Id'].unique().tolist() #Creates an array with the 'Individual Id' elements
    for y in non_repeated_id:
        ind_id_group=hh_group.groupby('Individual Id')
        individual_id_num=ind_id_group['Individual Id'].get_group(y).count()#Counts the times the 'y' Individual Id is repeated in the 'x' household
        if individual_id_num>1: #if this condition is true means that the 'y' Individual Id is repeated for the household 'x'
            exception=df_nil_individuals[(df_nil_individuals['Household Code']==x) & (df_nil_individuals['Individual Id']==y)]# Extracts the rows where these condotopns are true
            size_=exception.shape[0]
            for z in range(size_-1):#Inside this for, the rows are merged
                row_1=exception.iloc[0,:] #the first row is selected
                row_2=exception.iloc[1,:] #The second row is selected
                exception.iloc[0,:]=row_1.combine_first(row_2) #These rows are combined with combine_first, and the resulting row, is stored in the first position
                #of the group of rows
                exception=exception.drop(exception.index[1],axis=0)#The second row is deleted, it wont be useful anymore.
                #If the rows are more than two the cycle will give the necessary iterations(Size_-1) until all the rows are merged 
            A.append(exception)#The result of the merge is stored in one array
                

df_exceptions=pd.concat(A)#Creates a dataframe with the values that now are corrected
df_exceptions=df_exceptions = df_exceptions[~df_exceptions.index.duplicated(keep='first')] #And drops the repeatead rows that result from the process


In [7]:
#Finally the values thet were corrected in the previous cell code are inserted into the dataframe 'df_nil_individuals'\
#replacing its corresponding old rows 
for i in range(df_nil_individuals.index.shape[0]):
    for j in range(df_exceptions.index.shape[0]):
        if df_nil_individuals.index[i]==df_exceptions.index[j]:
            df_nil_individuals.loc[i,:]=df_exceptions.loc[df_exceptions.index[j],:]


df_nil_individuals=df_nil_individuals.drop_duplicates(subset=['Household Code', 'Individual Id'], keep='first')#finally the repeated rows are deleted


In [8]:
df_nil_individuals
#df_nil_individuals.head() 
#df_nil_individuals.tail() 
#df_nil_individuals.loc[df_nil_individuals['Household Code']==22217445,:]

,Household Code,Area,Status Code,Status Description,Individual Id,Age,Meter,Contact Date,01/05,02/05,03/05,04/05,05/05,06/05,07/05,08/05,09/05,10/05,11/05,12/05,13/05,14/05,15/05,16/05,17/05,18/05,19/05,20/05,21/05,22/05,23/05,24/05,25/05,26/05,27/05,28/05,Total,Accumulated
0,21100506.0,Saudi Arabia,130,to be polled and produced (On Production),2.0,49,Nano,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,28.0,162.0
1,21102391.0,Saudi Arabia,130,to be polled and produced (Need Monitoring),4.0,60,Nano,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,28.0,55.0
2,21103029.0,Saudi Arabia,130,to be polled and produced (On Production),2.0,61,Nano,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,28.0,110.0
3,21103029.0,Saudi Arabia,130,to be polled and produced (On Production),3.0,25,Nano,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,28.0,111.0
4,21103029.0,Saudi Arabia,130,to be polled and produced (On Production),4.0,19,Nano,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,28.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,22216800.0,Saudi Arabia,110,to be installed (recruited) (Ready to be insta...,5.0,17,Nano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,***,***,***,***,***,R,N,N,N,N,N,N,6.0,6.0
466,22216051.0,Saudi Arabia,110,to be installed (recruited) (Ready to be insta...,2.0,50,Nano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,***,***,***,***,***,***,***,***,***,***,***,R,R,N,N,N,N,N,5.0,5.0
469,22217021.0,Saudi Arabia,110,to be installed (recruited) (Ready to be insta...,2.0,33,Nano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,***,***,***,***,***,R,N,N,N,N,N,5.0,5.0
472,22217445.0,Saudi Arabia,110,to be installed (recruited) (Ready to be insta...,1.0,122,Nano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,***,***,R,N,N,N,N,N,5.0,5.0
